In [1]:
import pinecone
import openai
import numpy as np
import os
from dotenv import load_dotenv
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
# Langchain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, LLMChain ,LLMCheckerChain
from langchain.callbacks import wandb_tracing_enabled
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts.few_shot import FewShotPromptTemplate

from typing import Optional
from langchain.chains import SimpleSequentialChain ,SequentialChain
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.agents import AgentType, initialize_agent,AgentExecutor
from langchain.tools import tool
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.schema import HumanMessage, AIMessage, ChatMessage

# wandb
import wandb 

c:\Users\51703\Documents\GitHub\SUTDWTH23\env\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# Load variables from the .env file
load_dotenv('../.env')

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_ENVIRONMENT= os.getenv("PINECONE_ENVIRONMENT")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# wandb
os.environ["LANGCHAIN_WANDB_TRACING"] = "false"
os.environ['WANDB_NOTEBOOK_NAME'] = 'test_llm_pinecone.ipynb'
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "Singlife"

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [4]:
model_name = "gpt-3.5-turbo-0613"
temperature = 0.0
llm = ChatOpenAI(model_name=model_name, temperature=temperature)
llm.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
 'kwargs': {'model_name': 'gpt-3.5-turbo-0613',
  'temperature': 0.0,
  'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}}}

In [5]:
json_schema= {
    "name": "estimate expiration date",
    "description": "Given ingrediants and their Quantity, estimate the expiration date of each ingrediant",
    "type": "object",
    "properties": {
      "list_of_ingrediants": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "Name": { "type": "string" },
            "Quantity": { "type": "integer" },
            "Expiration": { "type": "string", "format": "date" }
          },
          "required": ["Name", "Quantity", "Expiration"]
        }
      }
    },
    "required": ["list_of_ingrediants"]
  }
  

In [7]:
video_prompt = PromptTemplate(
    template="""You are an expiratity date estimator, given this list of inputs of food ingredients, estimate based on today date 2023-08-26 when the food ingredients will expire.\nmy inventory of ingrediants:{ingredients}""",
    input_variables= ["ingredients"]
)

In [11]:
ingredients = {
    "Egg":12,
    "Apple":5,
    "Milk":2,
    "Bread":4,
    "Butter":3,
    "Cheese":2
}

In [9]:
expire_chain = create_structured_output_chain(json_schema, llm, video_prompt, verbose=True)

In [13]:
expire_chain.run(ingredients = ingredients)



> Entering new LLMChain chain...
Prompt after formatting:
You are an expiratity date estimator, given this list of inputs of food ingredients, estimate based on today date 2023-08-26 when the food ingredients will expire.
my inventory of ingrediants: {'Egg': 12, 'Apple': 5, 'Milk': 2, 'Bread': 4, 'Butter': 3, 'Cheese': 2}

> Finished chain.


{'list_of_ingrediants': [{'Name': 'Egg',
   'Quantity': 12,
   'Expiration': '2023-09-02'},
  {'Name': 'Apple', 'Quantity': 5, 'Expiration': '2023-08-30'},
  {'Name': 'Milk', 'Quantity': 2, 'Expiration': '2023-08-28'},
  {'Name': 'Bread', 'Quantity': 4, 'Expiration': '2023-08-29'},
  {'Name': 'Butter', 'Quantity': 3, 'Expiration': '2023-08-28'},
  {'Name': 'Cheese', 'Quantity': 2, 'Expiration': '2023-08-28'}]}